In [16]:
using BasisMatrices
using QuantEcon
using Plots
plotlyjs( )

Plots.PlotlyJSBackend()

In [17]:
# set environment
n = 8
smin = 0.7
smax = 1.3
maxit = 1000
# num_nodes is for gaussian quadrature
num_nodes = 3
tol = 0.0001


# make collocation matrix
sgrid0 = linspace(smin, smax, n)
basis = Basis(SplineParams(sgrid0, 0, 3), SplineParams(sgrid0, 0, 3))
S, (coordx, coordy) = nodes(basis)
Φ = BasisMatrix(basis, Expanded(), S, 0)

BasisMatrix{BasisMatrices.Expanded} of order [0 0]

In [18]:
mutable struct CPG
    alpha::Array{Float64,1}
    beta::Array{Float64,1}
    gamma::Array{Float64,1}
    sigma::Array{Float64,1}
    psi::Float64
    delta::Float64
end


function cost( model::CPG, s::Array{Float64, 2}, p::Int64)
    return model.beta[1]+( model.beta[2] ./ s(:,p))
end

cost (generic function with 1 method)

In [19]:
function prof( model::CPG, cost::Array{Float64, 2})
    return (model.alpha[1]-2*cost(1)+cost(2).^2)/(9*model.alpha[2])
end

prof (generic function with 1 method)

In [20]:
function reward( model::CPG, prof::Array{Float64, 2},x::Array{Float64, 2}, p::Int64)     
    return prof-(model.gamma[1]*x[:,p]+0.5*model.gamma(2)*x[:,p].^2)
end

reward (generic function with 1 method)

In [21]:
function capital( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2},p::Int64)     
    return (1-psi)*s + x
end

capital (generic function with 1 method)

In [22]:
# derivatives of  payoff, transition functions
# reward function

function rewardx( model::CPG, x::Array{Float64, 2}, p::Int64) 
    return -(model.gamma[1]+model.gamma[2]*x(:,p))
end

rewardx (generic function with 1 method)

In [23]:
function rewardxx( model::CPG, x::Array{Float64, 2}, p::Int64) 
    return -(model.gamma[2])
end

rewardxx (generic function with 1 method)

In [24]:
# transition function
function capitalx( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2},p::Int64) 
    return ones(n,1)
end

capitalx (generic function with 1 method)

In [25]:
function capitalxx( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2},p::Int64) 
    gxx = zeros(n,ds,dx,dx)
end

capitalxx (generic function with 1 method)

In [26]:
# Gaussian Quadrature
function gq(model::CPG, num_nodes::Int64)
    return qnwlogn([num_nodes, num_nodes], [0,0], diagm([model.sigma[1]^2, model.sigma[2]^2]))
end

gq (generic function with 1 method)

In [27]:
# collocation function
function vmax(model::CPG, colnodes::Array{Float64, 2}, b, action::Array{Float64, 2}, coef::Array{Float64, 2}, epss::Array{Float64, 2}, weights::Array{Float64, 1})
    xnew = action
    v = zeros((size(colnodes)[1], 2))
    for p in 1:2
        xl, xu = 0.0, colnodes[:, p]
        order1 = [0 0]
        order1[1, p] = 1
        order2 = [0 0]
        order2[1, p] = 2
        for it in 1:maxit
            util, util_der1, util_der2 = reward(model, colnodes, action, p), rewardx(model, colnodes, action, p), rewardxx(model, colnodes, action, p) 
            Ev, Evx, Evxx = 0.0, 0.0, 0.0
            for k in 1:num_nodes^2
                eps, weight= epss[k, :], weights[k]
                transition, transition_der1, transition_der2 = capital(model, action, eps), capitalx(model, action, eps, p), capitalxx(model, action, eps, p)
                vn = funeval(coef[:, p], b, transition)
                vnder1 =  funeval(coef[:, p], b, transition, order1)
                vnder2 = funeval(coef[:, p], b, transition, order2)
                Ev += weight * vn
                Evx += weight* vnder1.* transition_der1
                Evxx += weight * (vnder1.*transition_der2 + vnder2 .* (transition_der1.^2))
            end
            v[:, p] = util + Ev
            delx = -(util_der1 + model.delta * Evx) ./ (util_der2 + model.delta*Evxx)
            delx = min.(max.(delx, xl-action[:, p]), xu-action[:, p])
            action[:, p] = action[:, p] + delx
            if norm(delx) < tol
                break
            end
        end
        xnew[:, p] = action[:, p]
    end
    return v, xnew
end

vmax (generic function with 1 method)

In [28]:
# set initials
# num_nodesが3ならsrand(1234)
srand(12347)
initial = rand((size(Φ.vals[1])[1]), 2)
x = zeros((size(Φ.vals[1])[1]), 2)+1
v = zeros((size(Φ.vals[1])[1]), 2)
model = CPG([8.0,4.0], [1.8, 0.2],[0.4,3.0], [0.1,0.1], 0.1, 0.9)
e, w = gq(model, num_nodes)


([0.840965 0.840965; 1.0 0.840965; … ; 1.0 1.18911; 1.18911 1.18911], [0.0277778, 0.111111, 0.0277778, 0.111111, 0.444444, 0.111111, 0.0277778, 0.111111, 0.0277778])

In [30]:
# iteration for coefficients
c = initial
for it in 1:3
    cold = c
    vnew, x = vmax[model, S, basis, x, c, e, w]
    c = Φ.vals[1] \ vnew
    v = vnew
    if norm(cold - c) < tol
        break
    end
end

LoadError: [91mMethodError: no method matching getindex(::#vmax, ::CPG, ::Array{Float64,2}, ::BasisMatrices.Basis{2,Tuple{BasisMatrices.SplineParams{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}},BasisMatrices.SplineParams{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}}}}, ::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,2}, ::Array{Float64,1})[39m